In [148]:
poligon1 = [[5,1,5], [1,1,1], [5,1,5]]
poligon2 = [[1,1], [1,0]]
poligon3 = [[1,1,1], [0,1,0]]

poligons = [poligon1, poligon2, poligon3]

matrix = [[1,1,1,0,0,0,0,0],
          [0,1,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0]]

In [149]:
def print_matrix(matrix):
    for i in range(len(matrix)):
        print(matrix[i],'\n')

print_matrix(matrix)


[1, 1, 1, 0, 0, 0, 0, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 



In [150]:
mat_dublicated = matrix
for i in range(len(matrix[0])): #разобраться с тем, где какая координата
    for j in range(len(matrix)):
        mat_dublicated = matrix
        exit = False
        # условия, что объект влезает
        if len(poligon3) + i <= len(matrix) and len(poligon3[0]) + j <= len(matrix[0]):
            # распологаем объект
            for p in range(len(poligon3)):
                for k in range(len(poligon3[0])):
                    if (i+p) < len(matrix) and (j+k)< len(matrix[0]): #ещё с координатами матриц согласовать
                        # решаем есть ли пересечение и если нет, то размещаем
                        if mat_dublicated[i+p][j+k] + poligon3[p][k] < 2:
                            mat_dublicated[i+p][j+k] = mat_dublicated[i+p][j+k] + poligon3[p][k] #заменить на j-k когда будем упаковывать вниз
                        else:
                            exit = True
                            break
                if exit == True:
                    break
            print_matrix(mat_dublicated)
            print('--------------')
        if exit == False:
            matrix = mat_dublicated
            break
    if exit == False:
        break

[1, 1, 1, 0, 0, 0, 0, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

--------------
[1, 1, 1, 0, 0, 0, 0, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

--------------
[1, 1, 1, 0, 0, 0, 0, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

--------------
[1, 1, 1, 1, 1, 1, 0, 0] 

[0, 1, 0, 0, 1, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

--------------
